### Scrape GEMP
* Implement proof-of-concept scraper

In [1]:
## 0. Prerequisites
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4 as bs
import hidden
# used to write completed games to csv file
from csv import DictWriter

In [2]:
def get_gemp_tables(driver, tables):
    # <The Dudley Boyz: GET THE TABLE(S)! />
    
    
    keys = ['Recorded','TableID','Format','Tournament','Status','Players','GameLink','Winner']
        
    playing_table_body = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[3]/div[1]/div[4]/table/tbody')
    playing_entries = playing_table_body.find_elements_by_tag_name('tr')
    
    finished_table_body = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[3]/div[1]/div[6]/table/tbody')
    finished_entries = finished_table_body.find_elements_by_tag_name('tr')
 
    
    ## Update/Add rows from Playing Tables
    for i in range(1, len(playing_entries)):
        cols = playing_entries[i].find_elements_by_tag_name('td')
        table_id = playing_entries[i].get_attribute("class")
        
        if table_id not in tables:
            if DEBUG: print("adding Playing Table: "+table_id)
            row = {}
            row['Recorded'] = 0
            row[keys[1]]= table_id
            row[keys[2]]= cols[0].text
            row[keys[3]]= cols[1].text
            row[keys[4]]= cols[2].text
            row[keys[5]]= cols[3].text
            if len(playing_entries[i].find_elements_by_tag_name('a')) > 0:
                row[keys[6]]= playing_entries[i].find_elements_by_tag_name('a')[0].get_attribute("href")
            row[keys[7]]= 'NA'
            # Add table only if 'Format' is 'Open'
            if(row['Format'] == 'Open'):
                tables[table_id] = row
    
    # check length of tables dictionary
    if DEBUG: print(f'length of tables after Playing Tables: {len(tables)}')
    
    ## Updates tables with rows from Finished Tables
    for i in range(1, len(finished_entries)):
        cols = finished_entries[i].find_elements_by_tag_name('td')
        table_id = finished_entries[i].get_attribute("class")
        
#         print(f'row {i} of {len(finished_entries)-1} with table_id of {table_id}')
        if ((table_id in tables) & (len(cols) > 0)):
            if tables[table_id]['Recorded'] == 0:
                if DEBUG: print("updating Finished Table: "+table_id)
#                 table = tables[table_id]
                tables[table_id]['Status'] = cols[2].text
                tables[table_id]['Winner'] = cols[4].text
        else:
            if DEBUG: print("adding Finished Table: "+table_id)
            row = {}
            row['Recorded'] = 0
            row[keys[1]]= table_id
            row[keys[2]]= cols[0].text
            row[keys[3]]= cols[1].text
            row[keys[4]]= cols[2].text
            row[keys[5]]= cols[3].text
            row[keys[6]]= 'NA'
            row[keys[7]]= cols[4].text
            # Add table only if 'Format' is 'Open'
            if(row['Format'] == 'Open'):
                tables[table_id] = row
            
    # check length of tables dictionary
    if DEBUG: print(f'length of tables after Finished Tables: {len(tables)}')
    
    return tables
    
    
def record_gemp_tables(tables):
    headers = ['Recorded','TableID','Format','Tournament','Status','Players','GameLink','Winner']
    table_ids = list(tables.keys())
    
    
    # Check tables dictionay keys
    if DEBUG: print(f'keys in tables are {table_ids}')
    
    for table_id in table_ids:
#         print(table_id, tables[table_id])
        if ((tables[table_id]['Status'] == 'Finished') & (tables[table_id]['Format'] == 'Open') & (tables[table_id]['Recorded'] == 0)):
            if DEBUG: print(table_id, tables[table_id]) 
            with open('gemp_completed_games.csv', 'a', newline='') as f:
                dictwriter_object = DictWriter(f, fieldnames=headers)
                # Pass the data in the dictionary as an argument into the writerow() function
                tables[table_id]['Recorded'] = time.strftime('%Y-%m-%d %H:%M:%S%z (%Z)')
                dictwriter_object.writerow(tables[table_id])
                # Close the file object
                f.close()
            if DEBUG: print(f"recorded {tables[table_id]}")
    if DEBUG: print(f"length of tables after recording games: {len(tables)}")        
    recorded = True
    return recorded, tables


In [3]:
def login_gemp(driver):
    secrets = hidden.secrets()
    driver.get ("https://gemp.starwarsccg.org/gemp-swccg/")
    driver.find_element_by_id("login").send_keys(secrets["user"])
    driver.find_element_by_id("password").send_keys(secrets["pass"])
    driver.find_element_by_id("password").send_keys(Keys.RETURN)
    return driver

def listen_for_gemp_tables(driver):
    listening = True
    tables = {}
    sleep_time = 120 # length of time between cycles
    while(listening):
        tables = get_gemp_tables(driver, tables)
        recorded, tables = record_gemp_tables(tables)
        
        # ADD CHECK FOR NEW GEMP-TABLES
        # CREATE NEW WEBDRIVERs to WATCH AND RECORD RELEVANT ACTIVE GAMES
        # CREATE STRUCTURE FOR SIMULTANEOUS GAMES
        
        if recorded:
            if DEBUG: print(f"sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    return 0


def access_gemp(site_URL):
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver = login_gemp(driver)
    driver.get(site_URL)
    listen_for_gemp_tables(driver)
    
    source = driver.page_source
#     driver.quit()
    return source
  

In [4]:
# get gemp data
DEBUG = True
if DEBUG: print(f'Debug is ON')
site_URL = 'https://gemp.starwarsccg.org/gemp-swccg/hall.html'
source = access_gemp(site_URL)

Debug is ON
length of tables after Playing Tables: 0
length of tables after Finished Tables: 0
keys in tables are []
length of tables after recording games: 0
sleeping for 120 seconds...
adding Playing Table: table9418
adding Playing Table: table9424
adding Playing Table: table9427
adding Playing Table: table9432
adding Playing Table: table9433
adding Playing Table: table9430
adding Playing Table: table9439
adding Playing Table: table9437
length of tables after Playing Tables: 5
adding Finished Table: table9414
adding Finished Table: table9417
length of tables after Finished Tables: 5
keys in tables are ['table9424', 'table9427', 'table9432', 'table9430', 'table9439']
length of tables after recording games: 5
sleeping for 120 seconds...
adding Playing Table: table9418
adding Playing Table: table9433
adding Playing Table: table9437
length of tables after Playing Tables: 5
length of tables after Finished Tables: 5
keys in tables are ['table9424', 'table9427', 'table9432', 'table9430', 't

KeyboardInterrupt: 